## 기본 환경 설정

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!wget https://www.python.org/ftp/python/3.6.9/Python-3.6.9.tgz
!tar xvfz Python-3.6.9.tgz
!Python-3.6.9/configure
!make
!sudo make install

In [ ]:
!python --version

Python 3.6.9


In [ ]:
!pip install mxnet

In [ ]:
!pip install gluonnlp pandas tqdm

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install pandas

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

- torch, nn, F, optim : PyTorch 라이브러리에서 제공되는 모듈, 함수
- Dataset : 데이터 저장, DataLoader : 데이터 관리 및 공급
- gluonnlp : GlunoNLP, 자연어 처리 라이브러리
- numpy : NumPy 라이브러리, 다차원 배열과 행렬 연산에 이용
- tqdm : 진행 상황을 시각적으로 보여주는 라이브러리
- pandas : 데이터 조작 및 분석 라이브러리

In [ ]:
# KoBERT
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

- kobert_tokenizer : 한국어 텍스트를 위한 토크나이저
- BertModel : 구글에서 개발된 BERT 모델, 자연어 처리 모델 라이브러리
- AdamW : [Adam 알고리즘 + 가중치 감쇠] 모델의 복잡도를 줄임
- get_cosine_schedule_with_warmup : 학습률을 조절하는 스케줄러

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


결과가 0이 뜬다면, 런타임 유형 GPU로 변경해주기

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

- GPU : CPU 보다 빠르게 연산되는 병렬 프로세서
- cuda:0 : 첫 번째 GPU를 의미

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

- tokenizer : 문장을 토큰으로 분리하여, 인덱스로 변환
- bertmodel : 텍스트의 의미 이해, 특성 추출
- vocal : KoBERT의 Vocabulary 설정, 인덱스 매핑

## 데이터셋 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/dataset/dataset.xlsx')

In [ ]:
dataset.sample(n=1)

,문장,감정
1568,전세계적으로 대립이 극도로 치닫는구나,공포


In [ ]:
len(dataset)

96865

In [ ]:
dataset["감정"].value_counts()

슬픔    25545
분노    16082
공포    15901
놀람    15702
행복    13376
혐오     5429
중립     4830
Name: 감정, dtype: int64

In [ ]:
# 정수 변환
# 슬픔(0), 분노(1), 공포(2), 놀람(3), 행복(4), 혐오(5), 중립(6)
dataset.loc[(dataset["감정"] == "슬픔"), "감정"] = 0
dataset.loc[(dataset["감정"] == "분노"), "감정"] = 1
dataset.loc[(dataset["감정"] == "공포"), "감정"] = 2
dataset.loc[(dataset["감정"] == "놀람"), "감정"] = 3
dataset.loc[(dataset["감정"] == "행복"), "감정"] = 4
dataset.loc[(dataset["감정"] == "혐오"), "감정"] = 5
dataset.loc[(dataset["감정"] == "중립"), "감정"] = 6

In [ ]:
dataset["감정"].value_counts()

0    25545
1    16082
2    15901
3    15702
4    13376
5     5429
6     4830
Name: 감정, dtype: int64

In [ ]:
dataset.sample(n=10)

,문장,감정
96494,이번에 승진 시험에서 탈락해서 화가 나. 사실 내 점수가 더 높은데 동기 언니가 이...,0
55311,아무 일 없이 아빠가 내 이름을 부르면 내 성적 떨어진 거 이야기할까 봐 두렵고 무...,2
2998,사고나면 초음속으로 천국가겠군여ㅠ,2
35414,그 애비 인성으로보아 기업이 대를잇지 못할것,5
88764,나이 먹고 새 일자리를 구하려니 쉽지가 않네. 퇴직하고 몇 달을 쉬는 건지. 자식들...,0
6672,너무하네...,3
41312,어제 언니랑 싸웠는데 너무 심한 말을 한 것 같아서 후회돼. 순간적으로 너무 화가 ...,0
86224,늙고 병 드니까 찾아오는 친구나 사람들도 없고 정말 외로워. 젊을 때 사람들을 많이...,1
37215,집사서 혼자 살면되잖아 왜 저러지?,5
13610,목소리 크고 성격 가칠하면 살기 편한 나라!,1


In [ ]:
data_list = []
for q, label in zip(dataset['문장'], dataset['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)

['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네. 맞아. 하지만 그렇다고 아무나하고 결혼할 수도 없잖아.', '3']


In [ ]:
# train & test 데이터로 분류하기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size= 0.2, random_state = 0)

- test_size= 0.2 : 80%는 train, 20%는 test로 추출
- shuffle = True : 디폴트값으로 생략 가능
- random_state : 난수의 초기값 설정

In [ ]:
len(dataset_train)

77492

In [ ]:
len(dataset_test)

19373

### 입력 데이터 생성 (토큰화, 정수 인코딩, 패딩)

KoBERT 입력 데이터를 생성하기 위해 다음을 진행해야 한다.


토큰화 : 텍스트를 작은 단위로 나누는 과정

- "안녕하세요" : "안녕", "하세", "요"


정수 인코딩 : 토큰을 고유한 정수로 변환하는 과정

- "안녕" : 1, "하세" : 2, "요" : 3


패딩 : 지정된 입력 크기보다 짧다면, 토큰을 추가하는 과정
- 입력 크기가 10, 입력 문장은 7개의 토큰이라면 3개의 패딩 토큰을 추가하여 10개의 토큰으로 만든다.


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

- dataset : 원본 데이터셋
- sent_idx : 문장(sentence)의 인덱스
- label_idx : 라벨의 인덱스
- bert_tokenizer : BERT 토크나이저
- vocab : Vocaburary 설정
- max_len : 문장의 최대 길이
- pad : 패딩의 여부
- pair : 문장 쌍 여부

In [ ]:
# 파라미터 설정
max_len = 256
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
tok = tokenizer.tokenize

# BERTDataset(데이터셋, 문장 인덱스, 라벨 인덱스, 토크나이저, 어휘, 최대 길이, 패딩 여부, 문자쌍 여부)
# 토큰화, 정수 인코딩, 패딩 과정 진행
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 3238, 6410, 4627, 5330, 1370, 6897,  517, 6756, 6756, 6166,
        5400, 1168, 6855,   54, 1435, 2110, 3943, 7088, 4955,  881, 3093,
        5777, 5591, 2497, 7788,  880, 5876,   54, 1435,  517, 6249, 3943,
        7868, 7261, 2050, 5439,   54, 3945, 5859, 3942, 3647, 6099, 2872,
        3278,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
data_test[0]

(array([   2, 1435,  891, 3177, 7086, 2397, 7418, 5330, 3135, 1771, 5439,
        5007, 4619, 6441, 4012, 6398, 6116, 2426, 7852, 4998,   54, 1458,
        3239, 7788, 2856, 6527, 7848,   54, 4044, 2393, 5889, 3177, 2397,
        6896,  758, 7864, 6887, 1435,  891, 4832, 5859, 6824, 7086, 4004,
        7096, 3135, 1771, 5439, 4924, 5839,   54, 2397, 3475, 4332, 2233,
        2872, 3860, 2270, 7088, 4529, 3175, 6364, 6061, 4998,   54,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

1번 array는 패딩된 시퀀스, 1이라는 패딩 토큰 추가하여 길이 조정

2번 array는 패딩 전 원본 시퀀스의 길이와, 데이터 유형

3번 array는 어텐션 마스크 시퀀스

어텐션 마스크를 사용하여, 입력 시퀀스에서 주의해야 할 토큰과 무시해야 하는 토큰을 나타낸다. 어텐션 마스크는 0으로, 실제 연산에 필요한 토큰의 위치는 1로 채워진다.

In [ ]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBERT 모델

### BERT 기반의 감정 분류를 위한 클래스 정의

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

- num_classes : 예측할 감정 클래스의 수
- gen_attention_mask 메서드 : 토큰과 길이를 입력받아, 어텐션 마스크를 생성한다.
- forword 메서드 : BERT 모델에 입력을 전달하여, 어텐션 마스크를 적용한다. 그 후, 최종적으로 예측값을 반환한다.

In [ ]:
# BERT  모델
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

### Optimizer, Schedule 설정

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- no_decay : Weight Decay를 적용하지 않을 파라미터
- optimizer_grouped_parameters : 옵티마이저에 전달될 파라미터 그룹 정의
    - no_decay에 포함되지 않은 파라미터 : Weight Decay 0.01 적용
    - no_decay에 포함된 파라미터 : Weight Decay 적용 X
- optimizer : Adam 옵티마이저 이용
- loss_fn : 다중 클래스 분류에 사용되는 손실 함수(정답값과 예측값의 오차 계산)


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

- t_total : 훈련 데이터 로더의 총 배치 수
- warmup_step : 준비 단계 수
- scheduler : 코사인 스케줄러를 사용하여 학습률 조절

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

- calc_accuracy : 정확도 계산 함수, 예측 비율 계산

## 학습

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-43-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2422 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.1876771450042725 train acc 0.0625
epoch 1 batch id 201 loss 1.8496274948120117 train acc 0.19402985074626866
epoch 1 batch id 401 loss 1.4716947078704834 train acc 0.2821072319201995
epoch 1 batch id 601 loss 1.3116453886032104 train acc 0.36278078202995007
epoch 1 batch id 801 loss 1.2322205305099487 train acc 0.4154962546816479
epoch 1 batch id 1001 loss 0.9432114958763123 train acc 0.4499562937062937
epoch 1 batch id 1201 loss 1.0433194637298584 train acc 0.473199417152373
epoch 1 batch id 1401 loss 0.6931989192962646 train acc 0.48971716630977874
epoch 1 batch id 1601 loss 0.9332753419876099 train acc 0.5036500624609619
epoch 1 batch id 1801 loss 1.0261342525482178 train acc 0.5159112992781788
epoch 1 batch id 2001 loss 0.7755824327468872 train acc 0.524066091954023
epoch 1 batch id 2201 loss 0.8765082359313965 train acc 0.5330531576556111
epoch 1 batch id 2401 loss 1.10923433303833 train acc 0.5403217409412745
epoch 1 train acc 0.5411849710982659


<ipython-input-43-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/606 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6516922124904798


  0%|          | 0/2422 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2039860337972641 train acc 0.90625
epoch 5 batch id 201 loss 0.3708183765411377 train acc 0.8403296019900498
epoch 5 batch id 401 loss 0.581292986869812 train acc 0.8373597256857855
epoch 5 batch id 601 loss 0.6194708943367004 train acc 0.8407341930116472
epoch 5 batch id 801 loss 0.28135010600090027 train acc 0.8462858926342073
epoch 5 batch id 1001 loss 0.4098229706287384 train acc 0.8500874125874126
epoch 5 batch id 1201 loss 0.2794564664363861 train acc 0.8523365945045795
epoch 5 batch id 1401 loss 0.15981259942054749 train acc 0.8539213062098501
epoch 5 batch id 1601 loss 0.11835893243551254 train acc 0.8557346970643348
epoch 5 batch id 1801 loss 0.42367133498191833 train acc 0.8584640477512493
epoch 5 batch id 2001 loss 0.1525522619485855 train acc 0.8584770114942529
epoch 5 batch id 2201 loss 0.3783653974533081 train acc 0.8595666742389823
epoch 5 batch id 2401 loss 0.6760826110839844 train acc 0.8598630778842149
epoch 5 train acc 0.8598498142031379


  0%|          | 0/606 [00:00<?, ?it/s]

epoch 5 test acc 0.6522118558009647


In [49]:
# 학습 모델 저장
PATH = '/content/drive/MyDrive/colab/'
torch.save(model, PATH + 'KoBERT.pt')
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')